In [20]:
import json
import os

import hashlib
from typing import List, Dict, Any
from tqdm import tqdm
import sys
import concurrent.futures
import random

from get_text_embedding import get_text_embedding

from dotenv import load_dotenv
from openai import OpenAI

In [1]:
%pip install -q sentence-transformers rank_bm25 jieba

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 3.41.0 requires pillow<11.0,>=8.0, but you have pillow 11.3.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\liuch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
%pip install -U torch torchvision torchaudio

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 9.8 MB/s eta 0:04:10
     ---------------------------------------- 0.0/2.4 GB 28.4 MB/s eta 0:01:27
     ---------------------------------------- 0.0/2.4 GB 27.7 MB/s eta 0:01:29
     ---------------------------------------- 0.0/2.4 GB 27.0 MB/s eta 0:01:31
     ---------------------------------------- 0.0/2.4 GB 27.0 MB/s eta 0:01:31
     ---------------------------------------- 0.0/2.4 GB 26.9 MB/s eta 0:01:31
     ---------------------------------------- 0.0/2.4 GB 26.7 MB/s eta 0:01:32
     ---------------------------------------- 0.0/2.4 GB 26.4 MB/s eta 0:01:33
     ---------------------------------------- 0.0/2.4 GB 28.5 MB/s eta 0:01:26
     ---------------------------------------- 0.0/2.4 GB 26.2 MB/s eta 0:01:34
     ---------------------------------------- 0.0/2.4 GB 26.2 MB/s eta 0:01:34
  

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 95, in res

   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.4 MB 9.9 MB/s eta 0:00:25
   ---------------------------------------- 3.0/241.4 MB 37.8 MB/s eta 0:00:07
   - -------------------------------------- 7.2/241.4 MB 57.7 MB/s eta 0:00:05
   -- ------------------------------------- 12.9/241.4 MB 93.9 MB/s eta 0:00:03
   --- ----------------------------------- 18.9/241.4 MB 131.2 MB/s eta 0:00:02
   ---- ---------------------------------- 25.4/241.4 MB 165.0 MB/s eta 0:00:02
   ----- --------------------------------- 31.5/241.4 MB 162.4 MB/s eta 0:00:02
   ------ -------------------------------- 37.8/241.4 MB 162.4 MB/s eta 0:00:02
   ------ -------------------------------- 42.7/241.4 MB 131.2 MB/s eta 0:00:02
   ------- ------------------------------- 48.8/241.4 MB 129.5 MB/s eta 0:00:02
   -------- ------------------------------ 53.6/241.4 MB 131.2 MB/s eta 0:00:02
   --------- ----------------------------- 58.7/241.4

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\liuch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
# 1) Install/Imports (run once; skip install if already available)
# %pip install -q sentence-transformers rank_bm25 jieba

from sentence_transformers import CrossEncoder
from rank_bm25 import BM25Okapi
import jieba, re, numpy as np


C:\Users\liuch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0816 17:03:03.603000 25096 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
C:\Users\liuch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [12]:
# 2) Reranker helpers

def zh_en_tokenize(text: str):
    """Lightweight tokenizer that works for zh/en mixed corpora."""
    text = re.sub(r'\s+', ' ', (text or '').strip())
    # For Chinese, char-level or jieba wordcut both work; try jieba, fallback to chars
    words = list(jieba.cut(text))
    # If jieba returns too few tokens (pure EN), fall back to whitespace split
    if len(words) < 3:
        words = text.lower().split()
    # If still tiny (e.g., numbers/symbols), use char-level
    if len(words) < 3:
        words = list(text)
    return words

class HybridReranker:
    """
    Combine: Cross-Encoder score + (optional) BM25 & retriever score.
    Keeps your chunk dicts intact: {'content', 'metadata': {'file_name','page'}, 'score'?}
    """
    def __init__(self, model_name="BAAI/bge-reranker-base", device=None,
                 w_ce=0.7, w_bm25=0.2, w_ret=0.1):
        self.ce = CrossEncoder(model_name, device=device)
        self.w_ce, self.w_bm25, self.w_ret = w_ce, w_bm25, w_ret

    @staticmethod
    def _z(x):
        x = np.asarray(x, dtype=float)
        return (x - x.mean()) / (x.std() + 1e-8) if len(x) else x

    def rerank(self, question: str, candidates: list):
        """
        candidates: list of dicts with keys:
          - 'content': str
          - 'metadata': {'file_name', 'page'}
          - optional 'score': initial retriever score (dense/BM25)
        returns: same list, sorted by combined score, with 'rerank_score' added
        """
        if not candidates:
            return []

        # --- BM25 over candidate pool (cheap, local) ---
        corpus_tokens = [zh_en_tokenize(c['content']) for c in candidates]
        bm25 = BM25Okapi(corpus_tokens)
        bm25_scores = bm25.get_scores(zh_en_tokenize(question))

        # --- Cross-Encoder scores (expensive but accurate) ---
        pairs = [(question, c['content']) for c in candidates]
        ce_scores = self.ce.predict(pairs)  # higher = more relevant

        # --- Initial retriever score if present ---
        ret_scores = np.array([c.get('score', 0.0) for c in candidates], dtype=float)

        # --- Normalize and combine ---
        ce_z = self._z(ce_scores)
        bm25_z = self._z(bm25_scores)
        ret_z = self._z(ret_scores) if np.any(ret_scores) else np.zeros_like(ce_z)

        combo = self.w_ce * ce_z + self.w_bm25 * bm25_z + self.w_ret * ret_z

        # Attach & sort
        for c, s_ce, s_bm, s_ret, s_combo in zip(candidates, ce_scores, bm25_scores, ret_scores, combo):
            c['ce_score'] = float(s_ce)
            c['bm25_score'] = float(s_bm)
            c['ret_score'] = float(s_ret)
            c['rerank_score'] = float(s_combo)

        return sorted(candidates, key=lambda x: x['rerank_score'], reverse=True)


In [13]:
class PageChunkLoader:
    def __init__(self, json_path: str):
        self.json_path = json_path
    def load_chunks(self) -> List[Dict[str, Any]]:
        with open(self.json_path, 'r', encoding='utf-8') as f:
            return json.load(f)

In [14]:
class EmbeddingModel:
    def __init__(self, batch_size: int = 64):
        self.api_key = os.getenv('LOCAL_API_KEY')
        self.base_url = os.getenv('LOCAL_BASE_URL')
        self.embedding_model = os.getenv('LOCAL_EMBEDDING_MODEL')
        self.batch_size = batch_size
        if not self.api_key or not self.base_url:
            raise ValueError('请在.env中配置LOCAL_API_KEY和LOCAL_BASE_URL')

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        return get_text_embedding(
            texts,
            api_key=self.api_key,
            base_url=self.base_url,
            embedding_model=self.embedding_model,
            batch_size=self.batch_size
        )

    def embed_text(self, text: str) -> List[float]:
        return self.embed_texts([text])[0]

In [15]:
class SimpleVectorStore:
    def __init__(self):
        self.embeddings = []
        self.chunks = []
    def add_chunks(self, chunks: List[Dict[str, Any]], embeddings: List[List[float]]):
        self.chunks.extend(chunks)
        self.embeddings.extend(embeddings)
    def search(self, query_embedding: List[float], top_k: int = 3) -> List[Dict[str, Any]]:
        from numpy import dot
        from numpy.linalg import norm
        import numpy as np
        if not self.embeddings:
            return []
        emb_matrix = np.array(self.embeddings)
        query_emb = np.array(query_embedding)
        sims = emb_matrix @ query_emb / (norm(emb_matrix, axis=1) * norm(query_emb) + 1e-8)
        idxs = sims.argsort()[::-1][:top_k]
        return [self.chunks[i] for i in idxs]

In [16]:
from typing import Dict, Any, List, Optional
import os
from openai import OpenAI

class SimpleRAG:
    def __init__(
        self,
        chunk_json_path: str,
        model_path: str = None,
        batch_size: int = 32,
        use_rerank: bool = False,
        candidate_k: int = 120,
        final_k: int = 5,
        reranker=None,  # expect your HybridReranker instance
    ):
        self.loader = PageChunkLoader(chunk_json_path)
        self.embedding_model = EmbeddingModel(batch_size=batch_size)
        self.vector_store = SimpleVectorStore()

        # Rerank controls
        self.use_rerank = use_rerank
        self.candidate_k = candidate_k
        self.final_k = final_k
        self.reranker = reranker

    def setup(self):
        print("加载所有页chunk...")
        chunks = self.loader.load_chunks()
        print(f"共加载 {len(chunks)} 个chunk")
        print("生成嵌入...")
        embeddings = self.embedding_model.embed_texts([c["content"] for c in chunks])
        print("存储向量...")
        self.vector_store.add_chunks(chunks, embeddings)
        print("RAG向量库构建完成！")

    def query(self, question: str, top_k: int = 3) -> Dict[str, Any]:
        q_emb = self.embedding_model.embed_text(question)
        results = self.vector_store.search(q_emb, top_k)
        return {"question": question, "chunks": results}

    def _build_context(self, items: List[Dict[str, Any]]) -> str:
        return "\n".join(
            [
                f"[文件名]{c['metadata']['file_name']} [页码]{c['metadata']['page']}\n{c['content']}"
                for c in items
            ]
        )

    def generate_answer(self, question: str, top_k: int = 3) -> Dict[str, Any]:
        """
        检索+大模型生成式回答，返回结构化结果
        """
        qwen_api_key = os.getenv("LOCAL_API_KEY")
        qwen_base_url = os.getenv("LOCAL_BASE_URL")
        qwen_model = os.getenv("LOCAL_TEXT_MODEL")
        if not qwen_api_key or not qwen_base_url or not qwen_model:
            raise ValueError("请在.env中配置LOCAL_API_KEY、LOCAL_BASE_URL、LOCAL_TEXT_MODEL")

        # ------ Retrieval (+ optional rerank) ------
        q_emb = self.embedding_model.embed_text(question)

        if self.use_rerank and self.reranker is not None:
            # Stage A: larger pool
            candidates = self.vector_store.search(q_emb, self.candidate_k)
            if candidates:
                # Stage B: rerank
                ranked = self.reranker.rerank(question, candidates)
                chunks = ranked[: self.final_k]
            else:
                chunks = []
        else:
            # Baseline
            chunks = self.vector_store.search(q_emb, top_k)

        # Build LLM context
        context = self._build_context(chunks)

        # ------ LLM call (unchanged) ------
        prompt = (
            "你是一名专业的金融分析助手，请根据以下检索到的内容回答用户问题。\n"
            "请严格按照如下JSON格式输出：\n"
            '{"answer": "你的简洁回答", "filename": "来源文件名", "page": "来源页码"}\n'
            f"检索内容：\n{context}\n\n问题：{question}\n"
            "请确保输出内容为合法JSON字符串，不要输出多余内容。"
        )

        client = OpenAI(api_key=qwen_api_key, base_url=qwen_base_url)
        completion = client.chat.completions.create(
            model=qwen_model,
            messages=[
                {"role": "system", "content": "你是一名专业的金融分析助手。"},
                {"role": "user", "content": prompt},
            ],
            temperature=0.2,
            max_tokens=1024,
        )

        import json as pyjson
        from extract_json_array import extract_json_array

        raw = completion.choices[0].message.content.strip()
        json_str = extract_json_array(raw, mode="objects")

        if json_str:
            try:
                arr = pyjson.loads(json_str)
                if isinstance(arr, list) and arr:
                    j = arr[0]
                    answer = j.get("answer", "")
                    filename = j.get("filename", "")
                    page = j.get("page", "")
                else:
                    answer = raw
                    filename = chunks[0]["metadata"]["file_name"] if chunks else ""
                    page = chunks[0]["metadata"]["page"] if chunks else ""
            except Exception:
                answer = raw
                filename = chunks[0]["metadata"]["file_name"] if chunks else ""
                page = chunks[0]["metadata"]["page"] if chunks else ""
        else:
            answer = raw
            filename = chunks[0]["metadata"]["file_name"] if chunks else ""
            page = chunks[0]["metadata"]["page"] if chunks else ""

        # ------ Source fallback/override from best evidence (place it HERE) ------
        if chunks:
            best_file = chunks[0]["metadata"]["file_name"]
            best_page = chunks[0]["metadata"]["page"]
            # Fallback: only fill if model didn't provide values
            if not filename:
                filename = best_file
            if not page:
                page = best_page
            # If you prefer to ALWAYS trust retrieval/rerank for grading:
            # filename, page = best_file, best_page

        # Final return
        return {
            "question": question,
            "answer": answer,
            "filename": filename,
            "page": page,
            "retrieval_chunks": chunks,
        }


In [17]:
# 1) Imports & Paths
from pathlib import Path
import os, json, random
from tqdm.auto import tqdm
import concurrent.futures
import math

# Notebook is in .../notebook; project root is parent
NOTEBOOK_DIR = Path.cwd()
PROJ_ROOT = NOTEBOOK_DIR.parent

# Try common locations for train.json
CANDIDATE_TRAIN = [
    PROJ_ROOT / "datas" / "train.json",
    PROJ_ROOT / "data" / "train.json",
    NOTEBOOK_DIR / "datas" / "train.json",
    NOTEBOOK_DIR / "data" / "train.json",
]
TRAIN_PATH = next((p for p in CANDIDATE_TRAIN if p.exists()), None)
if TRAIN_PATH is None:
    raise FileNotFoundError(f"train.json not found in: {CANDIDATE_TRAIN}")

# Chunk JSON path (your earlier structure)
CHUNK_JSON_PATH = PROJ_ROOT / "notebook" / "sample_pdf_page_chunks.json"

# Outputs
EVAL_RAW_PATH = PROJ_ROOT / "eval_train_raw.json"
EVAL_SUMMARY_PATH = PROJ_ROOT / "eval_train_scored.json"

print("Notebook Dir:", NOTEBOOK_DIR)
print("Project Root :", PROJ_ROOT)
print("Train JSON   :", TRAIN_PATH)
print("Chunks JSON  :", CHUNK_JSON_PATH)


Notebook Dir: d:\Datawhale\Multimodal-RAG-Competitions\notebook
Project Root : d:\Datawhale\Multimodal-RAG-Competitions
Train JSON   : d:\Datawhale\Multimodal-RAG-Competitions\data\train.json
Chunks JSON  : d:\Datawhale\Multimodal-RAG-Competitions\notebook\sample_pdf_page_chunks.json


In [19]:
# ⚙️ 2) Initialize RAG
# from your_module.rag import SimpleRAG  # ← update to your actual import path

try:
    rag = SimpleRAG(str(CHUNK_JSON_PATH))
    rag.setup()
    print("RAG initialized.")
except NameError:
    raise NameError("SimpleRAG is not defined. Import your class (e.g., `from your_module.rag import SimpleRAG`).")


加载所有页chunk...
共加载 7802 个chunk
生成嵌入...


NameError: SimpleRAG is not defined. Import your class (e.g., `from your_module.rag import SimpleRAG`).

In [29]:
# Create your reranker beforehand
# reranker = HybridReranker(model_name="BAAI/bge-reranker-base")
reranker = HybridReranker(model_name="BAAI/bge-reranker-v2-m3")
rag = SimpleRAG(
    chunk_json_path=CHUNK_JSON_PATH,  # or your page/block chunk file
    use_rerank=True,
    candidate_k=120,
    final_k=5,
    reranker=reranker,
)
rag.setup()


C:\Users\liuch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liuch\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


加载所有页chunk...
共加载 7802 个chunk
生成嵌入...


Embedding: 100%|██████████| 244/244 [04:27<00:00,  1.10s/batch]

存储向量...
RAG向量库构建完成！


In [22]:
# 3) Load train and sample
with open(TRAIN_PATH, "r", encoding="utf-8") as f:
    train_data = json.load(f)

N = len(train_data)
random.seed(42)

sample_size = max(1, math.ceil(N * 0.10))
all_idx = list(range(N))
sample_idx = sorted(random.sample(all_idx, sample_size)) if sample_size < N else all_idx

print(f"Train size = {N} | Sample size = {len(sample_idx)}")
sample_idx[:10]


Train size = 118 | Sample size = 12


[3, 13, 14, 17, 28, 31, 35, 69, 81, 86]

In [23]:
# 4) Jaccard helper
def jaccard_char(a: str, b: str) -> float:
    a = (a or "").strip()
    b = (b or "").strip()
    if not a and not b:
        return 1.0
    set_a, set_b = set(a), set(b)
    union = set_a | set_b
    inter = set_a & set_b
    return len(inter) / len(union) if union else 0.0


In [30]:
# 5) Inference
def run_one(idx):
    item = train_data[idx]
    q = item.get("question", "")
    tqdm.write(f"[{sample_idx.index(idx)+1}/{len(sample_idx)}] {q[:60]}...")
    pred = rag.generate_answer(q, top_k=5)
    return idx, pred

results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as ex:
    for out in tqdm(ex.map(run_one, sample_idx), total=len(sample_idx), desc="Infer on train sample"):
        results.append(out)

# Save raw (idx, pred) for debugging
with open(EVAL_RAW_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"Saved raw predictions to: {EVAL_RAW_PATH}")


[1/12] 联邦制药的UBT37034在超重/肥胖适应症方面取得了哪些临床前数据？...
[2/12] 根据华创证券对凌云股份的深度研究报告，请问该公司在2024年的主要产品收入占比是多少？...
[3/12] 关于凌云股份（600480）的热冲压技术应用和发展前景，能否详细解释热冲压成型工艺与冷冲压成型工艺的主要区别？...
[4/12] 关于凌云股份（600480）的德国WAG业务板块及客户情况，请问具体有哪些主要客户？...
[5/12] 广联达的数字施工业务在2020年的资金压力如何？与同行业其他企业相比，其资金压力有何特点？...
[6/12] 如何评估广联达在数字化转型过程中面临的挑战及其应对策略？...
[7/12] 如何评估广联达在数字化转型中的竞争优势？...
[8/12] 如何分析广联达（002410.SZ）在2021年的PS估值水平及其与可比公司的差异？...
[9/12] 千味央厨的餐饮大客户经营数据在2022年第三季度有何变化？...
[10/12] 千味央厨公司在2020年的毛利率受原材料价格波动影响如何？...


Infer on train sample:   0%|          | 0/12 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\liuch\AppData\Local\Temp\jieba.cache
Loading model cost 0.788 seconds.
Prefix dict has been built successfully.
Infer on train sample:   0%|          | 0/12 [23:53<?, ?it/s]

[11/12] 关于伊利股份的历史发展和市场竞争，请问在2005年至2013年间，伊利如何通过创新产品和营销策略实现营收突破100亿大关...


Infer on train sample:   0%|          | 0/12 [25:59<?, ?it/s]

[12/12] 广联达（002410）的数字设计业务在2021年下半年将如何推进？...


Infer on train sample: 100%|██████████| 12/12 [35:25<00:00, 177.16s/it]  

Saved raw predictions to: d:\Datawhale\Multimodal-RAG-Competitions\eval_train_raw.json


## 6) Scoring vs Ground Truth
Score per item:
- page_match: 1 if exact page equals, else 0 (×0.25)
- filename_match: 1 if exact filename equals, else 0 (×0.25)
- answer_jaccard: char Jaccard (×0.5)


In [31]:
# 6) Score predictions
idx2pred = {idx: pred for idx, pred in results}

scored_rows = []
for idx in sample_idx:
    gt = train_data[idx]
    pred = idx2pred.get(idx, {})

    gt_q = gt.get("question", "")
    gt_a = gt.get("answer", "")
    gt_f = gt.get("filename", "")
    gt_p = gt.get("page", "")

    pr_a = pred.get("answer", "")
    pr_f = pred.get("filename", "")
    pr_p = pred.get("page", "")

    page_match = 1.0 if str(pr_p) == str(gt_p) else 0.0
    filename_match = 1.0 if str(pr_f) == str(gt_f) else 0.0
    answer_sim = jaccard_char(str(pr_a), str(gt_a))

    score = 0.25 * page_match + 0.25 * filename_match + 0.5 * answer_sim

    scored_rows.append({
        "idx": idx,
        "question": gt_q,
        "gt_answer": gt_a,
        "pr_answer": pr_a,
        "gt_filename": gt_f,
        "pr_filename": pr_f,
        "gt_page": gt_p,
        "pr_page": pr_p,
        "page_match": page_match,
        "filename_match": filename_match,
        "answer_jaccard": answer_sim,
        "score": score,
    })

# Sort by score ascending to inspect weak cases first
scored_rows_sorted = sorted(scored_rows, key=lambda x: x["score"])

with open(EVAL_SUMMARY_PATH, "w", encoding="utf-8") as f:
    json.dump(scored_rows_sorted, f, ensure_ascii=False, indent=2)

print(f"Saved scored results to: {EVAL_SUMMARY_PATH}")
print(f"max score: {max(r['score'] for r in scored_rows_sorted)}")
print(f"Mean score: {sum(r['score'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"min score: {min(r['score'] for r in scored_rows_sorted) }")
print(f"Mean Jaccard: {sum(r['answer_jaccard'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"Filename exact@1: {sum(r['filename_match'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")
print(f"Page exact@1: {sum(r['page_match'] for r in scored_rows_sorted)/len(scored_rows_sorted):.4f}")


Saved scored results to: d:\Datawhale\Multimodal-RAG-Competitions\eval_train_scored.json
max score: 0.5732323232323233
Mean score: 0.2638
min score: 0.09668508287292818
Mean Jaccard: 0.3193
Filename exact@1: 0.4167
Page exact@1: 0.0000


Note: With BM25 rerank :
max score: 0.5732323232323233  
Mean score: 0.2638   
min score: 0.09668508287292818  
Mean Jaccard: 0.3193  
Filename exact@1: 0.4167  
Page exact@1: 0.0000  

In [32]:
# Show a couple of worst and best cases inline (adjust k as needed)
k = 1
print("— Worst cases —")
for r in scored_rows_sorted[:k]:
    print("\nScore:", r["score"])
    print("Q:", r["question"])
    print("GT:", r["gt_answer"])
    print("PR:", r["pr_answer"])
    print("GT file/page:", r["gt_filename"], r["gt_page"])
    print("PR file/page:", r["pr_filename"], r["pr_page"])

print("\n— Best cases —")
for r in scored_rows_sorted[-k:]:
    print("\nScore:", r["score"])
    print("Q:", r["question"])
    print("GT:", r["gt_answer"])
    print("PR:", r["pr_answer"])
    print("GT file/page:", r["gt_filename"], r["gt_page"])
    print("PR file/page:", r["pr_filename"], r["pr_page"])


— Worst cases —

Score: 0.09668508287292818
Q: 如何评估广联达在数字化转型过程中面临的挑战及其应对策略？
GT: 广联达在数字化转型过程中面临的挑战主要包括以下几个方面：

1. **劳动力问题**：
   - 工人的体力劳动量较大，施工工作环境较苦，收入较低，导致施工人员流动性大。
   - 老龄化现象严重，30岁及以上比例的工人占比逐年提升，接近8成。
   - 受教育程度低，建筑施工以简单重复性工作为主，难以提供良好的职业上升通道。

2. **项目变化频繁**：
   - 建筑项目的设计、预算和时间表可能随时发生变化，影响工期与成本。
   - 变
PR: 广联达在数字化转型中面临业务扩展、内部系统整合及用户增长等挑战，应对策略包括分阶段推进云转型（从0-1到10-N）、建立客户成功指标体系驱动流程优化、开发BIM5D等技术产品，并通过数据中台整合数据资产，同时拓展供应链金融等增值服务。
GT file/page: 广联达-再谈广联达当前时点下如何看待其三条增长曲线-220217131页.pdf 32
PR file/page: 广联达-造价云化成果显著数字施工行稳致远-22112854页.pdf 24

— Best cases —

Score: 0.5732323232323233
Q: 联邦制药的UBT37034在超重/肥胖适应症方面取得了哪些临床前数据？
GT: 根据图片中的文字内容，联邦制药的UBT37034在超重/肥胖适应症方面的临床前数据如下：

1. UBT37034在饮食诱导肥胖大鼠（DIO Rats）上的临床前数据表明，21天给药后，UBT37034联用替尔泊肽减重13.6%。

2. 相比之下，Petrelintide联用替尔泊肽减重13.6%（-9.38%），Cagrilintide联用替尔泊肽减
PR: 联邦制药的UBT37034在饮食诱导肥胖大鼠（DIO Rats）临床前试验中，21天给药后联用替尔泊肽减重13.6%，优于Petrelintide（-9.38%）、Cagrilintide（-10.89%）及替尔泊肽单药（-3.02%）效果。
GT file/page: 联邦制药-港股公司研究报告-创新突破三靶点战略联姻诺和诺德-25071225页.pdf 11
PR file/pag

### Notes
- Set a different **sample fraction** by changing the `0.10` in `math.ceil(N * 0.10)`.
- If `filename`/`page` in ground truth differ in minor formatting (e.g., case, spaces), add normalization before comparison.
- You can plug this same scorer later for validation on a dev split.
